# Fathead Minnow

In [1]:
from helper_knn import *

db = pd.read_csv('lc_db_processed.csv').drop(columns = 'Unnamed: 0')

db = pd.concat([db,
                pd.DataFrame(pd.DataFrame(db['pubchem2d'].values).\
                             apply(lambda x: x.str.replace('', ' ').str.strip().str.split(' '), 
                                                                        axis = 1)[0].to_list(),
                   columns = ['pub'+ str(i) for i in range(1,882)])],
               axis = 1)

species = pd.read_csv('C:/Users/Simone/Desktop/Utilità tesi magistrale/data/species.txt', sep = '\|', engine = 'python')

fm = species[species['common_name'] == 'Fathead Minnow'][
    ['class', 'tax_order', 'family', 'genus', 'species']].values.ravel()

X_train = db.loc[(db[
    ['class', 'tax_order', 'family', 'genus', 'species']] == fm).all(axis =1)].copy().reset_index(drop=True)
y_train = multiclass_encoding(X_train['conc1_mean'].copy())

X_test = db.loc[~(db[
    ['class', 'tax_order', 'family', 'genus', 'species']] == fm).all(axis = 1)].copy().reset_index(drop=True)
y_test = multiclass_encoding(X_test['conc1_mean'].copy())

categorical = ['test_cas', 'fish', 'smiles', 'pubchem2d', 'conc1_mean', 'obs_duration_mean',
               'conc1_type', 'exposure_type', 'control_type', 'media_type',
               'application_freq_unit', 'species', 'class', 'tax_order', 'family', 'genus']
X_train.drop(columns = categorical, inplace = True)
X_test.drop(columns = categorical, inplace = True)

# KNN già fatto
file FM & RT -- multiclass

# Random forest

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterSampler

hyper_params_tune = {'max_depth' : [i for i in range(10,20,2)],
                     'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 1000, num = 11)],
                     'min_samples_split': [2, 5, 10],
                     'min_samples_leaf': [1, 2, 4]}

params_comb = list(ParameterSampler(hyper_params_tune, n_iter = 100, random_state = 52))

best_acc = 0

best_sens_micro = 0
best_sens_macro = 0
best_sens_weight = 0

best_precs_micro = 0
best_precs_macro = 0
best_precs_weight = 0

best_f1s_micro = 0
best_f1s_macro = 0
best_f1s_weight = 0


best_p = dict()
print(ctime())
for i in range(0, len(params_comb)):
    
    rfc = RandomForestClassifier(n_jobs = -1)

    for k,v in params_comb[i].items():
        setattr(rfc, k, v)
        
    rfc.fit(X_train, y_train)
    y_pred = rfc.predict(X_test)
    
    
    acc = accuracy_score(y_test, y_pred)
    
    sens_micro = recall_score(y_test, y_pred, average = 'micro')
    sens_macro = recall_score(y_test, y_pred, average = 'macro')
    sens_weight = recall_score(y_test, y_pred, average = 'weighted')
    
    precs_micro = precision_score(y_test, y_pred, average = 'micro')
    precs_macro = precision_score(y_test, y_pred, average = 'macro')
    precs_weight = precision_score(y_test, y_pred, average = 'weighted')
    
    f1s_micro = f1_score(y_test, y_pred, average = 'micro')
    f1s_macro = f1_score(y_test, y_pred, average = 'macro')
    f1s_weight = f1_score(y_test, y_pred, average = 'weighted')
    
    if acc > best_acc:
        best_p = params_comb[i]
        best_acc = acc

        best_sens_micro = sens_micro
        best_sens_macro = sens_macro
        best_sens_weight = sens_weight

        best_precs_micro = precs_micro
        best_precs_macro = precs_macro
        best_precs_weight = precs_weight

        best_f1s_micro = f1s_micro
        best_f1s_macro = f1s_macro
        best_f1s_weight = f1s_weight

print(ctime())
print(f'Model: {best_p}')

print('Accuracy:            ', best_acc, end = '\n\n')

print('Micro Recall:        ', best_sens_micro)
print('Macro Recall:        ', best_sens_macro)
print('Weighted Recall:     ', best_sens_weight, end = '\n\n')

print('Micro Precision:     ', best_precs_micro)
print('Macro Precision:     ', best_precs_macro)
print('Weighted Precision:  ', best_precs_weight, end = '\n\n')

print('Micro F1:            ', best_f1s_micro)
print('Macro F1:            ', best_f1s_macro)
print('Weighted F1:         ', best_f1s_weight)

Thu May  6 10:23:14 2021
Thu May  6 10:33:56 2021
Model: {'n_estimators': 920, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 18}
Accuracy:             0.5407561567811308

Micro Recall:         0.5407561567811308
Macro Recall:         0.5437252411290127
Weighted Recall:      0.5407561567811308

Micro Precision:      0.5407561567811308
Macro Precision:      0.5467809366610823
Weighted Precision:   0.5468030020360778

Micro F1:             0.5407561567811308
Macro F1:             0.5427808293939542
Weighted F1:          0.5413474693649254


# Simple RASAR

In [2]:
from sklearn.linear_model import LogisticRegression

def right_neighbor(neighbors, X_train, X_train_i):
    # IDX Neighbors
    idx_neigh_0 = pd.DataFrame(neighbors[1])[0].apply(lambda x: X_train_i.iloc[x].name)
    idx_neigh_1 = pd.DataFrame(neighbors[1])[1].apply(lambda x: X_train_i.iloc[x].name)
    
    idx_neigh = idx_neigh_0.copy()
    
    # dove l'indice del primo vicino risulta essere uguale a se stesso lo sostituisco con il secondo vicino
    idx_neigh[X_train.index == idx_neigh_0] = idx_neigh_1[X_train.index == idx_neigh_0].values
    
    # Distance from the Nearest Neighbor that is NOT itself
    dist_0 = pd.DataFrame(neighbors[0])[0]
    dist_1 = pd.DataFrame(neighbors[0])[1]
    
    distance = dist_0.copy()
    distance[X_train.index == idx_neigh_0] = dist_1[X_train.index == idx_neigh_0].values
    
    return idx_neigh, distance

In [4]:
X_train0 = X_train[y_train == 0].copy()
X_train1 = X_train[y_train == 1].copy()

print(ctime())
## DF Train

knn0 = KNeighborsClassifier(n_neighbors = 2)
knn0.fit(X_train0, y_train[y_train == 0])

neigh_train0 = knn0.kneighbors(X_train, return_distance = True)
idx_neigh0, dist0 = right_neighbor(neigh_train0, X_train, X_train0)

knn1 = KNeighborsClassifier(n_neighbors = 2)
knn1.fit(X_train1, y_train[y_train == 1])

neigh_train1 = knn1.kneighbors(X_train, return_distance = True)
idx_neigh1, dist1 = right_neighbor(neigh_train1, X_train, X_train1)

df_rasar_train = pd.DataFrame({'idx_train': X_train.index.values,
                      'label_train': y_train,
                      'idx_neigh0': idx_neigh0.values,
                      'dist_neigh0': dist0,       
                      'idx_neigh1': idx_neigh1.values,
                      'dist_neigh1': dist1})

# DF test
knn0 = KNeighborsClassifier(n_neighbors = 1)
knn0.fit(X_train0, y_train[y_train == 0])
neigh_test0 = knn0.kneighbors(X_test, return_distance = True)
idx_neigh_test0 = pd.DataFrame(neigh_test0[1])[0].apply(lambda x: X_train0.iloc[x].name)

knn1 = KNeighborsClassifier(n_neighbors = 1)
knn1.fit(X_train1, y_train[y_train == 1])
neigh_test1 = knn1.kneighbors(X_test, return_distance = True)
idx_neigh_test1 = pd.DataFrame(neigh_test1[1])[0].apply(lambda x: X_train1.iloc[x].name)

df_rasar_test = pd.DataFrame({'idx_test': X_test.index.values,
              'label_test': y_test,
              'idx_neigh0': idx_neigh_test0.values,
              'dist_neigh0': neigh_test0[0].ravel(),
              'idx_neigh1': idx_neigh_test1.values,
              'dist_neigh1': neigh_test1[0].ravel()})




lrc = LogisticRegression()
lrc.fit(df_rasar_train[['dist_neigh0','dist_neigh1']], y_train)
y_pred = lrc.predict(df_rasar_test[['dist_neigh0','dist_neigh1']])

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

print(ctime())
print("Accuracy: ", accuracy_score(y_test, y_pred),
      "\n Sensitivity:", recall_score(y_test, y_pred),
      "\n Specificity:", tn/(tn+fp),
     "\n F1 score:", f1_score(y_test, y_pred))

Mon Feb 15 13:13:42 2021
Mon Feb 15 13:14:26 2021
Accuracy:  0.7449416117470228 
 Sensitivity: 0.4514851485148515 
 Specificity: 0.9477903793508017 
 F1 score: 0.5913301222675065


# Rainbow Trout

In [1]:
from helper_knn import *

db = pd.read_csv('lc_db_processed.csv').drop(columns = 'Unnamed: 0')

db = pd.concat([db,
                pd.DataFrame(pd.DataFrame(db['pubchem2d'].values).\
                             apply(lambda x: x.str.replace('', ' ').str.strip().str.split(' '), 
                                                                        axis = 1)[0].to_list(),
                   columns = ['pub'+ str(i) for i in range(1,882)])],
               axis = 1)

species = pd.read_csv('C:/Users/Simone/Desktop/Utilità tesi magistrale/data/species.txt', sep = '\|', engine = 'python')

fm = species[species['common_name'] == 'Rainbow Trout'][
    ['class', 'tax_order', 'family', 'genus', 'species']].values.ravel()

X_train = db.loc[(db[
    ['class', 'tax_order', 'family', 'genus', 'species']] == fm).all(axis =1)].copy().reset_index(drop=True)
y_train = multiclass_encoding(X_train['conc1_mean'].copy())

X_test = db.loc[~(db[
    ['class', 'tax_order', 'family', 'genus', 'species']] == fm).all(axis = 1)].copy().reset_index(drop=True)
y_test = multiclass_encoding(X_test['conc1_mean'].copy())

categorical = ['test_cas', 'fish', 'smiles', 'pubchem2d', 'conc1_mean', 'obs_duration_mean',
               'conc1_type', 'exposure_type', 'control_type', 'media_type',
               'application_freq_unit', 'species', 'class', 'tax_order', 'family', 'genus']
X_train.drop(columns = categorical, inplace = True)
X_test.drop(columns = categorical, inplace = True)

# KNN già fatto

# Random forest

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterSampler

hyper_params_tune = {'max_depth' : [i for i in range(10,20,2)],
                     'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 1000, num = 11)],
                     'min_samples_split': [2, 5, 10],
                     'min_samples_leaf': [1, 2, 4]}

params_comb = list(ParameterSampler(hyper_params_tune, n_iter = 100, random_state = 52))

best_acc = 0

best_sens_micro = 0
best_sens_macro = 0
best_sens_weight = 0

best_precs_micro = 0
best_precs_macro = 0
best_precs_weight = 0

best_f1s_micro = 0
best_f1s_macro = 0
best_f1s_weight = 0


best_p = dict()
print(ctime())
for i in range(0, len(params_comb)):
    
    rfc = RandomForestClassifier(n_jobs = -1)

    for k,v in params_comb[i].items():
        setattr(rfc, k, v)
        
    rfc.fit(X_train, y_train)
    y_pred = rfc.predict(X_test)
    
    
    acc = accuracy_score(y_test, y_pred)
    
    sens_micro = recall_score(y_test, y_pred, average = 'micro')
    sens_macro = recall_score(y_test, y_pred, average = 'macro')
    sens_weight = recall_score(y_test, y_pred, average = 'weighted')
    
    precs_micro = precision_score(y_test, y_pred, average = 'micro')
    precs_macro = precision_score(y_test, y_pred, average = 'macro')
    precs_weight = precision_score(y_test, y_pred, average = 'weighted')
    
    f1s_micro = f1_score(y_test, y_pred, average = 'micro')
    f1s_macro = f1_score(y_test, y_pred, average = 'macro')
    f1s_weight = f1_score(y_test, y_pred, average = 'weighted')
    
    if acc > best_acc:
        best_p = params_comb[i]
        best_acc = acc

        best_sens_micro = sens_micro
        best_sens_macro = sens_macro
        best_sens_weight = sens_weight

        best_precs_micro = precs_micro
        best_precs_macro = precs_macro
        best_precs_weight = precs_weight

        best_f1s_micro = f1s_micro
        best_f1s_macro = f1s_macro
        best_f1s_weight = f1s_weight

print(ctime())
print(f'Model: {best_p}')

print('Accuracy:            ', best_acc, end = '\n\n')

print('Micro Recall:        ', best_sens_micro)
print('Macro Recall:        ', best_sens_macro)
print('Weighted Recall:     ', best_sens_weight, end = '\n\n')

print('Micro Precision:     ', best_precs_micro)
print('Macro Precision:     ', best_precs_macro)
print('Weighted Precision:  ', best_precs_weight, end = '\n\n')

print('Micro F1:            ', best_f1s_micro)
print('Macro F1:            ', best_f1s_macro)
print('Weighted F1:         ', best_f1s_weight)

Thu May  6 10:41:04 2021
Thu May  6 10:53:49 2021
Model: {'n_estimators': 680, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 14}
Accuracy:             0.5485811577752554

Micro Recall:         0.5485811577752554
Macro Recall:         0.5451273416416709
Weighted Recall:      0.5485811577752554

Micro Precision:      0.5485811577752554
Macro Precision:      0.5579716936603097
Weighted Precision:   0.548133450787278

Micro F1:             0.5485811577752554
Macro F1:             0.5431980499649429
Weighted F1:          0.5404587464268362


# Simple RASAR

In [3]:
X_train0 = X_train[y_train == 0].copy()
X_train1 = X_train[y_train == 1].copy()

print(ctime())
## DF Train

knn0 = KNeighborsClassifier(n_neighbors = 2)
knn0.fit(X_train0, y_train[y_train == 0])

neigh_train0 = knn0.kneighbors(X_train, return_distance = True)
idx_neigh0, dist0 = right_neighbor(neigh_train0, X_train, X_train0)

knn1 = KNeighborsClassifier(n_neighbors = 2)
knn1.fit(X_train1, y_train[y_train == 1])

neigh_train1 = knn1.kneighbors(X_train, return_distance = True)
idx_neigh1, dist1 = right_neighbor(neigh_train1, X_train, X_train1)

df_rasar_train = pd.DataFrame({'idx_train': X_train.index.values,
                      'label_train': y_train,
                      'idx_neigh0': idx_neigh0.values,
                      'dist_neigh0': dist0,       
                      'idx_neigh1': idx_neigh1.values,
                      'dist_neigh1': dist1})

# DF test
knn0 = KNeighborsClassifier(n_neighbors = 1)
knn0.fit(X_train0, y_train[y_train == 0])
neigh_test0 = knn0.kneighbors(X_test, return_distance = True)
idx_neigh_test0 = pd.DataFrame(neigh_test0[1])[0].apply(lambda x: X_train0.iloc[x].name)

knn1 = KNeighborsClassifier(n_neighbors = 1)
knn1.fit(X_train1, y_train[y_train == 1])
neigh_test1 = knn1.kneighbors(X_test, return_distance = True)
idx_neigh_test1 = pd.DataFrame(neigh_test1[1])[0].apply(lambda x: X_train1.iloc[x].name)

df_rasar_test = pd.DataFrame({'idx_test': X_test.index.values,
              'label_test': y_test,
              'idx_neigh0': idx_neigh_test0.values,
              'dist_neigh0': neigh_test0[0].ravel(),
              'idx_neigh1': idx_neigh_test1.values,
              'dist_neigh1': neigh_test1[0].ravel()})




lrc = LogisticRegression()
lrc.fit(df_rasar_train[['dist_neigh0','dist_neigh1']], y_train)
y_pred = lrc.predict(df_rasar_test[['dist_neigh0','dist_neigh1']])

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

print(ctime())
print("Accuracy: ", accuracy_score(y_test, y_pred),
      "\n Sensitivity:", recall_score(y_test, y_pred),
      "\n Specificity:", tn/(tn+fp),
     "\n F1 score:", f1_score(y_test, y_pred))

Mon Feb 15 13:16:30 2021
Mon Feb 15 13:17:14 2021
Accuracy:  0.8043700340522134 
 Sensitivity: 0.685207456404089 
 Specificity: 0.8766411378555798 
 F1 score: 0.7256228607816604
